In [1]:
# import the necessary packages
from imutils.video import FileVideoStream    #FileVideoStream for slower speed   #VideoStream for higher speed
from pyzbar import pyzbar
import datetime
import imutils
import time
import cv2

# initialize the video stream 
print("[INFO] starting video stream...")
vs = FileVideoStream("./videos/test2.mp4").start()   

# open the output CSV file for writing and initialize the set of
# barcodes found thus far
csv = open("./output_files/results1.csv", "w")
csv.write("Time Stamp,Barcode Data,Barcode Type\n")
found = set()
found_time = set()
count = 0

try:
    # loop over the frames from the video stream
    while True:  
        # grab the frame from the threaded video stream and resize it to
        # have a maximum width of x pixels
        frame = vs.read()
        frame = imutils.resize(frame, width = 1225)   #width = x pixels
    
        # find the barcodes in the frame and decode each of the barcodes
        barcodes = pyzbar.decode(frame)

    # loop over the detected barcodes
        for barcode in barcodes:
            # extract the bounding box location of the barcode and draw
            # the bounding box surrounding the barcode on the image
            (x, y, w, h) = barcode.rect
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

            # the barcode data is a bytes object so if we want to draw it
            # on our output image we need to convert it to a string first
            barcodeData = barcode.data.decode("utf-8")
            barcodeType = barcode.type

            # draw the barcode data and barcode type on the image
            text = "{} ({})".format(barcodeData, barcodeType)
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                
            # if the barcode text is currently not in our CSV file, write
            # the timestamp + barcode data + barcode type to disk and update the set
            if barcodeData not in found:
                csv.write("{},{},{}\n".format(datetime.datetime.now(), barcodeData, barcodeType))
                csv.flush()
                found.add(barcodeData)
                #print(found)    #for printing stored barcode values
                
        # show the output frame
        cv2.imshow("Barcode Scanner", frame)
        #cv2.imwrite("./frames/frame({}).jpg".format(count),frame)   #for saving the video frame by frame
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
        count = count + 1

except AttributeError :    #exception to handle the end of the video
        print("End of video")
        
# close the output CSV file do a bit of cleanup    
print("[INFO] cleaning up...")
csv.close()
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
End of video
[INFO] cleaning up...
